In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.preprocessing import image


dataset_path = '/kaggle/input/trashnet/dataset-resized'

all_images = []
labels = []

for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for file in os.listdir(category_path):
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(category_path, file)
                try:
                    img = cv2.imread(image_path)
                    if img is None:
                        print(f"Skipping corrupted image: {image_path}")
                        continue
                    all_images.append(image_path)
                    labels.append(category)
                except Exception as e:
                    print(f"Skipping corrupted image {image_path}: {e}")
                    continue

print(f"Total images found: {len(all_images)}")
print(f"Categories: {set(labels)}")
print(f"Image distribution: {dict((label, labels.count(label)) for label in set(labels))}")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Split into train (80%) and test (20%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, labels, test_size=0.2, random_state=42, stratify=labels
)

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

train_labels_cat = to_categorical(train_labels_encoded)
test_labels_cat = to_categorical(test_labels_encoded)



print(f"Training set: {len(train_images)} images ({len(train_images)/len(all_images)*100:.1f}%)")
print(f"Testing set: {len(test_images)} images ({len(test_images)/len(all_images)*100:.1f}%)")

print("\nLabel distribution in training set:")
for label in set(train_labels):
    print(f"  {label}: {train_labels.count(label)}")

print("\nLabel distribution in test set:")
for label in set(test_labels):
    print(f"  {label}: {test_labels.count(label)}")

# Augmetation

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as keras_image

def augment_and_load_images(image_paths, labels, target_size, augment=True, target_per_class=500):
    """
    Load and optionally augment images for TrashNet dataset.
    Ensures each class has exactly target_per_class images through augmentation.
    
    Args:
        image_paths: List of image file paths
        labels: List of corresponding labels
        target_size: Target image size (height, width)
        augment: Whether to apply data augmentation
        target_per_class: Target number of images per class (default: 500)
    
    Returns:
        Tuple of (images_array, labels_array)
    """
    if augment:
        datagen = ImageDataGenerator(
            fill_mode='nearest',
            horizontal_flip=True,
            shear_range=0.2,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            zoom_range=0.2
        )
        print(f"\n{'='*60}")
        print(f"LOADING TRAINING IMAGES WITH AUGMENTATION")
        print(f"Target: {target_per_class} images per class")
        print(f"{'='*60}")
    else:
        print(f"\n{'='*60}")
        print(f"LOADING IMAGES (NO AUGMENTATION)")
        print(f"{'='*60}")
    
    # Group images by class
    class_images = {}
    for img_path, label in zip(image_paths, labels):
        if label not in class_images:
            class_images[label] = []
        class_images[label].append(img_path)
    
    print(f"\nClass distribution (original):")
    for label, paths in sorted(class_images.items()):
        print(f"  Class {label}: {len(paths)} images")
    
    images_list = []
    labels_list = []
    
    # Process each class
    for label, img_paths in sorted(class_images.items()):
        print(f"\nProcessing class {label}...")
        num_original = len(img_paths)
        
        if augment and num_original < target_per_class:
            # Calculate how many augmentations needed per image
            augmentations_per_image = (target_per_class // num_original)
            remainder = target_per_class % num_original
            
            print(f"  Original images: {num_original}")
            print(f"  Target images: {target_per_class}")
            print(f"  Augmentations per image: {augmentations_per_image}")
        
        for idx, img_path in enumerate(img_paths):
            if (idx + 1) % 50 == 0 or (idx + 1) == num_original:
                print(f"  Progress: {idx + 1}/{num_original}")
            
            try:
                # Load and preprocess image
                img = keras_image.load_img(img_path, target_size=target_size)
                img_array = keras_image.img_to_array(img)
                img_array = img_array / 255.0  # Normalize to [0, 1]
                
                # Add original image
                images_list.append(img_array)
                labels_list.append(label)
                
                if augment and num_original < target_per_class:
                    # Determine number of augmentations for this specific image
                    num_augs = augmentations_per_image - 1  # -1 because we already added original
                    if idx < remainder:
                        num_augs += 1  # Add extra augmentation to first 'remainder' images
                    
                    if num_augs > 0:
                        img_array_for_aug = img_array * 255.0
                        img_array_expanded = np.expand_dims(img_array_for_aug, axis=0)
                        
                        aug_iterator = datagen.flow(
                            img_array_expanded,
                            batch_size=1,
                            shuffle=False,
                            seed=42 + idx + label * 10000  # Reproducible augmentations
                        )
                        
                        for _ in range(num_augs):
                            img_augmented = next(aug_iterator)[0]
                            img_augmented = img_augmented / 255.0  # Normalize back
                            images_list.append(img_augmented)
                            labels_list.append(label)
                        
            except Exception as e:
                print(f"  Warning: Failed to load {img_path}: {e}")
                continue
        
        # Count images for this class
        class_count = sum(1 for l in labels_list if l == label)
        print(f"  Final count for class {label}: {class_count} images")
    
    print(f"\n{'='*60}")
    print(f"FINAL DATASET SUMMARY")
    print(f"{'='*60}")
    print(f"Total images: {len(images_list)}")
    
    # Show final class distribution
    unique_labels = sorted(set(labels_list))
    print(f"\nFinal class distribution:")
    for label in unique_labels:
        count = sum(1 for l in labels_list if l == label)
        print(f"  Class {label}: {count} images")
    print()
    
    return np.array(images_list), np.array(labels_list)

In [ ]:
train_images_loaded, train_labels_augmented = augment_and_load_images(
    train_images,
    train_labels_encoded,
    target_size=(224, 224),
    augment=True,
    target_per_class=600
)


test_images_loaded, test_labels_final = augment_and_load_images(
    test_images,
    test_labels_encoded,
    target_size=(224, 224),
    augment=False,
    target_per_class=600
)

# Modeling

## Base model

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False



model = Sequential([
    base_model,
    GlobalAveragePooling2D()
])

## Feature Extraction

In [ ]:
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.efficientnet import preprocess_input
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
def extract_features_from_arrays(image_arrays, labels, model):
    """
    Extract features from pre-loaded image arrays.
    Args:
        image_arrays: numpy array of images (N, w, h, 3), values in [0, 1]
        labels: numpy array of labels
        model: Feature extraction model
    Returns:
        features: numpy array of features
        labels: same labels array
    """
    features_list = []
    total = len(image_arrays)
    
    print(f"\n{'='*60}")
    print(f"EXTRACTING FEATURES")
    print(f"{'='*60}")
    print(f"\nProcessing {total} images...")
    
    for idx, img_array in enumerate(image_arrays):
        if (idx + 1) % 100 == 0 or (idx + 1) == total:
            print(f"  Progress: {idx + 1}/{total} ({(idx+1)/total*100:.1f}%)")
        
        # Expand dimensions for batch processing
        img_array = np.expand_dims(img_array, axis=0)
        
        # Preprocess (scale back to [0, 255] then normalize)
        img_array_scaled = img_array * 255.0
        img_preprocessed = preprocess_input(img_array_scaled)
        
        # Extract features
        features = model.predict(img_preprocessed, verbose=0)
        features_list.append(features[0])
    
    features = np.array(features_list)
    
    print(f"\nFeature extraction complete!")
    print(f"Features shape: {features.shape}")
    print(f"Labels shape: {labels.shape}\n")
    
    return features, labels

In [ ]:
train_features, train_labels_final = extract_features_from_arrays(
    train_images_loaded,
    train_labels_augmented,
    model
)


test_features, test_labels_final = extract_features_from_arrays(
    test_images_loaded,
    test_labels_final,
    model
)

## Classification

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# param_grid = {
#     'C': [7.7,7.76,7.75,7.79],
#     'gamma': [0.001, 0.0005, 0.0001, 0.00005, 0.00001],
#     'kernel': ['rbf'],
# }


svm_classifier = SVC(
    kernel='rbf',           
    C=7.7,                 
    gamma='scale',          
    probability=True,       
    class_weight='balanced', 
    random_state=42
)

#grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=10, scoring='accuracy', verbose=2, n_jobs=-1)
#grid_search.fit(train_features, train_labels_final)
#print(f"Best parameters found: {grid_search.best_params_}")
#print(f"Best cross-validation accuracy: {grid_search.best_score_:.2f}")
#svm_classifier = grid_search.best_estimator_

svm_classifier.fit(train_features, train_labels_final)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


# knn_param_grid = {
#     'n_neighbors': [3, 5, 7, 9, 11],
#     'weights': ['uniform', 'distance'],
#     'metric': ['euclidean', 'cosine']
# }

# knn_grid_search = GridSearchCV(
#     estimator=KNeighborsClassifier(),
#     param_grid=knn_param_grid,
#     cv=5,
#     scoring='accuracy',
#     verbose=2,
#     n_jobs=-1
# )
# knn_grid_search.fit(train_features, train_labels_encoded)

knn_classifier = KNeighborsClassifier(
    n_neighbors=7,          
    weights='distance',     
    algorithm='auto',       
    metric='minkowski',    
    p=2,                    
    n_jobs=-1              
)
knn_classifier.fit(train_features, train_labels_final)

# print(f"Best KNN parameters: {knn_grid_search.best_params_}")
# print(f"Best KNN CV accuracy: {knn_grid_search.best_score_:.2f}")
# knn_classifier = knn_grid_search.best_estimator_

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("\n=== SVM Results ===")


svm_test_pred = svm_classifier.predict(test_features)
print(f"Test Accuracy: {accuracy_score(test_labels_final, svm_test_pred):.4f}")
print("\nClassification Report:")
print(classification_report(test_labels_final, svm_test_pred, target_names=label_encoder.classes_))

In [ ]:
print("\n=== KNN Results ===")

knn_test_pred = knn_classifier.predict(test_features)
print(f"Test Accuracy: {accuracy_score(test_labels_final, knn_test_pred):.4f}")
print("\nClassification Report:")
print(classification_report(test_labels_final, knn_test_pred, target_names=label_encoder.classes_))

# Testing

In [ ]:
from IPython.display import Image, display
from tensorflow.keras.applications.efficientnet import preprocess_input

def predict_image(image_path, show_image=True, confidence_threshold=0.5):
    if show_image:
        display(Image(image_path, width=224))
    
    # Load and preprocess image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (224, 224))
    
    # Normalize to [0, 1] (MATCH YOUR TRAINING PREPROCESSING)
    img_array = img_resized / 255.0
    img_array = np.expand_dims(img_array, axis=0).astype('float32')
    
    img_array_scaled = img_array * 255.0
    img_preprocessed = preprocess_input(img_array_scaled)
    
    # Extract features
    features = model.predict(img_preprocessed, verbose=0)
    
    # SVM prediction (use features directly)
    svm_pred = svm_classifier.predict(features)[0]
    svm_class = label_encoder.inverse_transform([svm_pred])[0]
    
    # KNN prediction (use features directly)
    knn_pred = knn_classifier.predict(features)[0]
    knn_class = label_encoder.inverse_transform([knn_pred])[0]
    
    # Get confidence scores (use features directly)
    if hasattr(svm_classifier, 'predict_proba'):
        svm_proba = svm_classifier.predict_proba(features)[0]
        svm_confidence = max(svm_proba)
        svm_confidence_pct = svm_confidence * 100
    else:
        svm_confidence = None
        svm_confidence_pct = "N/A"
    
    knn_proba = knn_classifier.predict_proba(features)[0]
    knn_confidence = max(knn_proba)
    knn_confidence_pct = knn_confidence * 100
    
    # Apply confidence threshold
    if svm_confidence is not None and svm_confidence < confidence_threshold:
        svm_class = "Unknown Object"
    if knn_confidence < confidence_threshold:
        knn_class = "Unknown Object"
    
    # Print results
    print("\n" + "="*50)
    print("PREDICTION RESULTS")
    print("="*50)
    print(f"\n🔹 SVM Prediction: {svm_class}")
    if svm_confidence_pct != "N/A":
        print(f"   Confidence: {svm_confidence_pct:.2f}%")
        if svm_confidence < confidence_threshold:
            print(f"   ⚠️  Below threshold ({confidence_threshold*100:.0f}%)")
    
    print(f"\n🔹 KNN Prediction: {knn_class}")
    print(f"   Confidence: {knn_confidence_pct:.2f}%")
    if knn_confidence < confidence_threshold:
        print(f"   ⚠️  Below threshold ({confidence_threshold*100:.0f}%)")
    
    print("="*50 + "\n")
    
    return svm_class, knn_class

# Test
svm_result, knn_result = predict_image(
    r"/kaggle/input/test-model/paper4.png",
    confidence_threshold=0.5
)

In [ ]:
import pickle
import json
from tensorflow.keras.models import Model

# ===================================
# SAVE ALL MODELS AND METADATA
# ===================================

print("Saving models and metadata...")

# 1. Save the feature extractor model (use SavedModel format, not H5)
import tensorflow as tf
tf.saved_model.save(model, '/kaggle/working/feature_extractor_model')
print("   ✓ Feature extractor saved as TensorFlow SavedModel")

# 2. Save SVM classifier
print("\n2. Saving SVM classifier...")
with open('/kaggle/working/svm_classifier.pkl', 'wb') as f:
    pickle.dump(svm_classifier, f)
print("   ✓ SVM classifier saved")

# 3. Save KNN classifier
print("\n3. Saving KNN classifier...")
with open('/kaggle/working/knn_classifier.pkl', 'wb') as f:
    pickle.dump(knn_classifier, f)
print("   ✓ KNN classifier saved")

# 4. Save label encoder
print("\n4. Saving label encoder...")
with open('/kaggle/working/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
print("   ✓ Label encoder saved")

# 5. Save metadata (useful information)
print("\n5. Saving metadata...")
metadata = {
    'classes': label_encoder.classes_.tolist(),
    'num_classes': len(label_encoder.classes_),
    'input_shape': (224, 224, 3),
    'feature_shape': train_features.shape[1],
    'svm_params': {
        'kernel': 'rbf',
        'C': 7.7,
        'gamma': 'scale',
        'probability': True,
        'class_weight': 'balanced'
    },
    'knn_params': {
        'n_neighbors': 7,
        'weights': 'distance',
        'metric': 'minkowski',
        'p': 2
    }
}

with open('/kaggle/working/model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print("   ✓ Metadata saved")

print("\n" + "="*60)
print("ALL MODELS SAVED SUCCESSFULLY!")
print("="*60)
print("\nSaved files:")
print("  1. /kaggle/working/feature_extractor_model/ (directory)")
print("  2. /kaggle/working/svm_classifier.pkl")
print("  3. /kaggle/working/knn_classifier.pkl")
print("  4. /kaggle/working/label_encoder.pkl")
print("  5. /kaggle/working/model_metadata.json")
print("\nDownload all these files from Kaggle to use in your application!")